In [1]:
%pip install ipywidgets
%pip install sentencetransformers
%pip install pyquery
%pip install tqdm
%pip install nltk
%pip install gensim

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sentencetransformers (from versions: none)
ERROR: No matching distribution found for sentencetransformers

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
from pyquery import PyQuery as pq
import requests
import asyncio
from typing import Any, AsyncGenerator, List, Set, Tuple, Dict
from urllib.parse import urljoin
import json
import unicodedata
from tqdm.notebook import tqdm

TIMES_URL = "https://time.com/"
TIMES_HOME_URL = urljoin(TIMES_URL, "section/us/")
TIMES_PAGE_URL = "/section/us/?page={page}"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:128.0) Gecko/20100101 Firefox/128.0"
}
N_ARTICLES = 20
SUMMARIZED_ARTICLES_PICKLE = "summarized_articles.pkl"
ARTICLES_CORPUS_PICKLE = "articles_with_corpus.pkl"
ARTICLES_PICKLE = "articles.pkl"
ARTICLES_TOPIC_PICKLE = "articles_with_topic.pkl"
CLUSTERS_PICKLE = "clusters.pkl"

In [3]:
def json_str(cls):
    def custom_str(self):
        class_name = self.__class__.__name__
        properties = {k: v for k, v in self.__dict__.items()}
        return json.dumps({"class": class_name, "properties": properties}, indent=4)
    cls.__str__ = custom_str
    cls.__repr__ = custom_str
    return cls

def remove_unicode_and_normalize(s: str) -> str:
    normalized = unicodedata.normalize('NFKD', s)
    return normalized.encode('ascii', 'ignore').decode('ascii')

In [4]:
@json_str
class RateLimiter:
    def __init__(self, n: int=1, interval:float=2):
        self.n = n
        self.interval = interval
        self.calls = 0
        self.lock = asyncio.Lock()

    async def acquire(self) -> None:
        async with self.lock:
            if self.calls >= self.n:
                await asyncio.sleep(self.interval)
                self.calls = 0
            self.calls += 1

In [5]:
from typing import TypeVar


T = TypeVar('T')
async def async_islice(generator: AsyncGenerator[T, None], stop: int, start: int = 0) -> AsyncGenerator[T, None]:
    i = -1
    async for value in generator:
        i += 1
        if i < start:
            continue
        if i >= stop:
            break
        yield value

In [6]:
@json_str
class Article:
    def __init__(self, url: str, title: str, paragraphs: List[str]):
        self.paragraphs: List[str] = paragraphs
        self.title: str = title
        self.url: str = url

In [7]:
async def iter_times(max_pages: int = 10) -> AsyncGenerator[Article, None]:
    rl = RateLimiter(interval=1)

    def get_page_from_url(url) -> pq:
        resp = requests.get(url, headers=HEADERS)
        if resp.status_code != 200:
            return None
        page = pq(resp.text)
        return page
    
    async def iter_pages(start_url: str) -> AsyncGenerator[pq, None]:
        page_num = 1
        
        def get_next_page() -> pq:
            nonlocal page_num
            page_num += 1
            page = get_page_from_url(urljoin(TIMES_URL, TIMES_PAGE_URL.format(page=page_num)))
            return page
        
        await rl.acquire()
        page = get_page_from_url(start_url)
        await rl.acquire()
        next_page = get_next_page()
        yield page

        while next_page is not None:
            yield page
            await rl.acquire()
            page = next_page
            next_page = get_next_page()

    def get_article_urls(head_page: str) -> List[str]:
        article_urls = [x.attr("href") for x in head_page("div.component div.taxonomy-tout a").items()]
        return article_urls
    
    def get_article_from_url(url: str) -> Article:
        page = get_page_from_url(url)
        try:
            title = remove_unicode_and_normalize(page("h1.self-baseline").text())
            paragraphs = [remove_unicode_and_normalize(x.text()) for x in page("p.self-baseline.px-0").items()]
            return Article(url, title, paragraphs)
        except Exception as e:
            print(f"Article at url='{url}' failed to parse: {e}")
            return None

    async for page in async_islice(iter_pages(TIMES_HOME_URL), max_pages):
        urls = get_article_urls(page)
        for url in urls:
            yield get_article_from_url(urljoin(TIMES_URL, url))
            await rl.acquire()
    

In [46]:
N_ARTICLES = 500

articles: List[Article] = []
with tqdm(total=N_ARTICLES) as pbar:
    async for article in async_islice(iter_times(max_pages=100), N_ARTICLES):
        pbar.update(1)
        if article is not None:
            articles.append(article)
    tqdm.write(f"Processed {len(articles)}/{N_ARTICLES} before iterator ended.")


import pickle
with open(ARTICLES_PICKLE, "wb") as f:
    pickle.dump(articles, f)

  0%|          | 0/500 [00:00<?, ?it/s]

Processed 500/500 before iterator ended.


In [8]:
import pickle
with open(ARTICLES_PICKLE, "rb") as f:
    articles: List[Article] = pickle.load(f)

In [9]:
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from nltk.probability import FreqDist


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

#https://stackoverflow.com/questions/43742956/fast-named-entity-removal-with-nltk
def fast_ne_removal(tokens):
    tagged = pos_tag(tokens)
    tree = ne_chunk(tagged)
    non_entities = [leaf[0] for leaf in tree if type(leaf) != nltk.Tree]
    return non_entities

def clean_text(text):
    text = re.sub(r'\d+', '', text)
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.isalnum() and word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    freq_dist = FreqDist(words)
    words = [word for word in words if freq_dist[word] > 1]
    
    words = fast_ne_removal(words)
    words = [s.lower() for s in words]
    return words

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gsuga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gsuga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gsuga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\gsuga\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\gsuga\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\gsuga\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-

In [10]:
@json_str
class SummarizedArticle(Article):    
    def __init__(self, summary: str, **kwargs):
        super().__init__(**kwargs)
        self.summary: str = summary

In [12]:
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
import numpy as np
# https://github.com/UKPLab/sentence-transformers/tree/master/examples/applications/text-summarization
from LexRank import degree_centrality_scores

SUMMARY_LENGTH = 3

model = SentenceTransformer("all-mpnet-base-v2", device="cuda")

summarized_articles: List[SummarizedArticle] = []

with tqdm(total=len(articles)) as pbar:
    for article in articles:
        sentences: List[str] = []
        for paragraph in article.paragraphs:
            if paragraph is not None and len(paragraph) > 0:
                sentences.extend([tok for tok in sent_tokenize(paragraph) if len(tok.strip()) > 0])

        sentences = np.array(sentences)

        if len(sentences) < 1:
            continue
        
        embeddings = model.encode(sentences)

        # https://github.com/UKPLab/sentence-transformers/blob/master/examples/applications/text-summarization/text-summarization.py
        similarity_scores = model.similarity(embeddings, embeddings).numpy()
        centrality_scores = degree_centrality_scores(similarity_scores, threshold=None)
        most_central_sentence_indices = np.argsort(-centrality_scores)

        summary = "\n".join(sentences[most_central_sentence_indices[0:SUMMARY_LENGTH]])
        summarized_articles.append(SummarizedArticle(summary, **article.__dict__))
        pbar.update(1)
    pbar.write(f"Summarized {len(summarized_articles)} articles.")

import pickle

with open(SUMMARIZED_ARTICLES_PICKLE, "wb") as f:
    pickle.dump(summarized_articles, f)

  0%|          | 0/500 [00:00<?, ?it/s]

Summarized 499 articles.


In [11]:
import pickle
with open(SUMMARIZED_ARTICLES_PICKLE, "rb") as f:
    summarized_articles: List[SummarizedArticle] = pickle.load(f)

In [12]:
print("\n\n".join([s.summary for s in summarized_articles[0:2]]))

Besides breaching the Trump campaign, officials also believe that Iran tried to hack into the presidential campaign of Kamala Harris.
We have observed increasingly aggressive Iranian activity during this election cycle, specifically involving influence operations targeting the American public and cyber operations targeting Presidential campaigns, said the statement released by the FBI, the Office of the Director of National Intelligence and the Cybersecurity and Infrastructure Security Agency.
Earlier this month, Microsoft issued a report detailing foreign agents attempts to interfere in this years election, citing an instance of an Iranian military intelligence unit in June sending a spear-phishing email to a high-ranking official of a presidential campaign from a compromised email account of a former senior advisor.

As the Democratic Party finds itself once again on the cusp of potentially electing its first female President, Clinton, now 76, used her speech to rally behind Vice Pre

In [13]:
@json_str
class ArticleWithCorpus(SummarizedArticle):
    def __init__(self, corpus: List[str], **kwargs):
        super().__init__(**kwargs)
        self.corpus : List[str] = corpus

In [14]:
articles_corpus: List[ArticleWithCorpus] = [ArticleWithCorpus(clean_text("\n".join(s.paragraphs)), **s.__dict__) for s in summarized_articles]

import pickle
with open(ARTICLES_CORPUS_PICKLE, "wb") as f:
    pickle.dump(articles_corpus, f)

In [14]:
import pickle
with open(ARTICLES_CORPUS_PICKLE, "rb") as f:
    articles_corpus: List[ArticleWithCorpus] = pickle.load(f)

In [15]:
from sentence_transformers import util, SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode([" ".join(a.corpus) for a in articles_corpus], batch_size=64, show_progress_bar=True, convert_to_tensor=True)
clusters: List[List[int]] = util.community_detection(embeddings, min_community_size=3, threshold=0.5)

import pickle
with open(CLUSTERS_PICKLE, "wb") as f:
    pickle.dump(clusters, f)

c:\Users\gsuga\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\gsuga\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

c:\Users\gsuga\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [15]:
import pickle
with open(CLUSTERS_PICKLE, "rb") as f:
    clusters: List[List[int]] = pickle.load(f)

In [16]:
for i, cluster in enumerate(clusters[0:3]):
    print(f"({i + 1}) {len(cluster)} Articles")
    for sentence_id in cluster[0:3]:
        print("\t", articles_corpus[sentence_id].title)
    print("\t...")

(1) 37 Articles
	 
	 How Republicans Are Reacting to the Colorado Ruling to Remove Trump From the Ballot
	 Read the Full Transcripts of Donald Trumps Interviews With TIME
	...
(2) 35 Articles
	 What Americas Student Photojournalists Saw at the Campus Protests
	 Columbias Relationship With Student Protesters Has Long Been Fraught
	 Gaza Calls, Columbia Falls: Campus Protesters Defy Suspension Threats and Occupy Hall
	...
(3) 12 Articles
	 High Debt Threatens the U.S. Economy
	 The Republican War on Food Programs
	 Is America in Decline?
	...


In [20]:
%pip install ollama

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [70]:
import ollama

modelspec='''
FROM llama3.1:70b
SYSTEM You will be given a number of article summaries you need to find the common topic between. This topic should only be a couple of words at a maximum and there should only be one topic. Make sure the topic corresponds to EVERY summary provided. If you cannot find a topic respond with only NONE. Summaries are separated by newlines. Respond only with the short topic as described with no extra information.
PARAMETER temperature 0.0
'''

ollama.create(model='llama3.1-70-topic', modelfile=modelspec)

{'status': 'success'}

In [71]:
@json_str
class ArticleWithTopic(ArticleWithCorpus):
    def __init__(self, topic: str, **kwargs):
        super().__init__(**kwargs)
        self.topic: str = topic

In [74]:
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
import numpy as np
# https://github.com/UKPLab/sentence-transformers/tree/master/examples/applications/text-summarization
from LexRank import degree_centrality_scores

topic_sentences: List[List[str]] = []
for cluster in clusters:
    sentences: List[str] = []
    for idx in cluster:
        sentences.extend(sent_tokenize(articles_corpus[idx].summary))
    topic_sentences.append(sentences)

TOPIC_SUMMARY_LENGTH = 10

model = SentenceTransformer("all-mpnet-base-v2", device="cuda")

summaries: List[str] = []
with tqdm(total=len(clusters), desc="Summarizing Clusters") as pbar:
    for i, sentences in enumerate(topic_sentences):
        sentences = np.array(sentences)
        if len(sentences) < 1:
            continue
        
        embeddings = model.encode(sentences)

        # https://github.com/UKPLab/sentence-transformers/blob/master/examples/applications/text-summarization/text-summarization.py
        similarity_scores = model.similarity(embeddings, embeddings).numpy()
        centrality_scores = degree_centrality_scores(similarity_scores, threshold=None)
        most_central_sentence_indices = np.argsort(-centrality_scores)

        summary = "\n\n".join(sentences[most_central_sentence_indices[0:TOPIC_SUMMARY_LENGTH]])
        summaries.append(summary)
        pbar.update(1)
    pbar.write(f"Summarized {len(summaries)} clusters.")



#summaries = ["\n\n\n".join([articles_corpus[sentence_id].summary.replace("\n", " ") for sentence_id in cluster[0:3]]) for cluster in clusters]

articles_topic: List[ArticleWithTopic] = []
with tqdm(total=len(summaries), desc="Extracting topics") as pbar:
    for i, summary in enumerate(summaries):
        topic = ollama.generate(model="llama3.1-70-topic", prompt=summary)['response']
        cluster = clusters[i]
        
        for sentence_id in cluster:
            articles_topic.append(ArticleWithTopic(topic, **articles_corpus[sentence_id].__dict__))

        pbar.update(1)

import pickle
with open(ARTICLES_TOPIC_PICKLE, "wb") as f:
    pickle.dump(articles_topic, f)

c:\Users\gsuga\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Summarizing Clusters:   0%|          | 0/3 [00:00<?, ?it/s]

Summarized 54 clusters.


Extracting topics:   0%|          | 0/54 [00:00<?, ?it/s]

In [75]:
import pickle
with open(ARTICLES_TOPIC_PICKLE, "rb") as f:
    articles_topic: List[ArticleWithTopic] = pickle.load(f)

In [76]:
%pip install pyvis

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [77]:
import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex

#chatgpt
def get_combined_colormap(N):
    colormaps = ['tab20', 'tab20c', 'tab10', 'Set3']  # A combination of discrete colormaps
    colors = []

    for cmap_name in colormaps:
        cmap = plt.cm.get_cmap(cmap_name)
        colors.extend([rgb2hex(c) for c in cmap.colors])

    return colors[:N] if N <= len(colors) else colors * (N // len(colors)) + colors[:N % len(colors)]
    #return [colors[i % len(colors)] for i in range(0, N)]

In [78]:
from nltk.probability import FreqDist
from pyvis.network import Network
from collections import defaultdict
from statistics import median

net = Network(height="100vh", width="100%", bgcolor="#222222", font_color="white", cdn_resources="remote", neighborhood_highlight=True)
net.toggle_physics(True)

all_words: List[str] = []

for at in articles_topic:
    if at.topic == "NONE":
        continue
    all_words.extend(at.corpus)

shared_words: List[str] = []
word_cluster_residency: List[str] = []
for cluster in clusters:
    cluster_words: List[str] = []
    for idx in cluster:
        ac = articles_corpus[idx]
        cluster_words.extend(ac.corpus)
    cluster_freq = FreqDist(cluster_words)
    shared_words.extend([word for word in cluster_words if cluster_freq[word] >= len(cluster) * .75])
    word_cluster_residency.extend(set(cluster_words))

word_cluster_count = FreqDist(word_cluster_residency)

topic_articles: Dict[str, List[ArticleWithTopic]] = defaultdict(lambda: [])
for article in articles_topic:
    l = topic_articles[article.topic]
    l.append(article)
    topic_articles[article.topic] = l

word_freq = FreqDist(all_words)
mean_freq = sum(word_freq.values()) / len(articles_corpus)
median_freq = median(word_freq.values())
print(f"Total Unique Words={len(set(all_words))}")
print(f"Mean Word Frequency Overall={mean_freq}")
print(f"Median Word Frequency Overall={median_freq}")
shared_words = list(set([word for word in shared_words if word_cluster_count[word] <= 3 and word in all_words]))

net.add_nodes(shared_words, size=[min(max(word_cluster_count[word], 1), 5) for word in shared_words])

edges: List[Tuple[str, str]] = []
colors = get_combined_colormap(len(topic_articles.items()))
if "NONE" in topic_articles:
    del topic_articles["NONE"]
i = 0
for topic, articles in topic_articles.items():
    color = colors[i]
    i += 1
    topic_words: Set[str] = set()
    for article in articles:
        topic_words = topic_words.union(article.corpus)
    for word in topic_words:
        if word in shared_words:
            edges.append((topic, word, color))

net.add_nodes(list(topic_articles.keys()), size=[30] * len(topic_articles.keys()), color=[colors[i] for i in range(0, len(topic_articles.items()))])
for k1, k2, color in edges:
    net.add_edge(source=k1, to=k2, color=color)

net.save_graph("net.html")
print("Open net.html in your browser to view.")

Total Unique Words=5081
Mean Word Frequency Overall=178.82565130260522
Median Word Frequency Overall=4
Open net.html in your browser to view.


C:\Users\gsuga\AppData\Local\Temp\ipykernel_21612\241976220.py:10: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap(cmap_name)
